In [9]:
import pandas as pd
import numpy as np
import sklearn.preprocessing
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn_pandas'

In [2]:
dfTrain = pd.read_csv('../data/trialPromoResults.csv', index_col='index')
# dfCustdb = pd.read_csv('../data/custdatabase.csv', index_col='index')
# dfActual = pd.read_csv('../data/Cust_Actual.csv', index_col='index')
# dfTrain = pd.read_csv('../data/Train.csv', index_col='index')

In [3]:
dfTrain.columns

Index(['sex', 'mstatus', 'age', 'children', 'occupation', 'education',
       'income', 'avbal', 'avtrans', 'decision'],
      dtype='object')

In [4]:
dfTrain.shape

(1000, 10)

In [14]:
# map columns to transformations
# outputting a dataframe
mapper = DataFrameMapper([
    ('sex',sklearn.preprocessing.LabelBinarizer()),
    ('mstatus',sklearn.preprocessing.LabelBinarizer()),
    (['age'],sklearn.preprocessing.StandardScaler()),
    ('occupation',sklearn.preprocessing.LabelBinarizer()),
    ('education',sklearn.preprocessing.LabelBinarizer()),
    (['income'],sklearn.preprocessing.StandardScaler()),
    (['avbal'],sklearn.preprocessing.StandardScaler()),
    (['avtrans'],sklearn.preprocessing.StandardScaler()),
    ('decision',sklearn.preprocessing.LabelBinarizer())
], df_out=True)

NameError: name 'DataFrameMapper' is not defined

In [12]:
np.round(mapper.fit_transform(data.copy()),2)

NameError: name 'mapper' is not defined

In [13]:
# list all columns with transformed names
mapper.transformed_names

NameError: name 'mapper' is not defined

In [15]:
from sklearn.neural_network import MLPClassifier


In [16]:
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)

In [17]:
x_train = mapper.drop('decision',axis=1)
y_train = mapper['decision']

NameError: name 'mapper' is not defined

In [18]:
mlp.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [19]:
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)